1. Отобразите все записи из таблицы `company` по компаниям, которые закрылись.

In [ ]:
SELECT *
FROM company
WHERE status = 'closed';

2. Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы `company`. Отсортируйте таблицу по убыванию значений в поле `funding_total`.

In [ ]:
SELECT funding_total
FROM company
WHERE country_code = 'USA' AND category_code = 'news'
ORDER BY funding_total DESC;

3. Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

In [ ]:
SELECT SUM(price_amount)
FROM acquisition
WHERE acquired_at BETWEEN '2011-01-01' AND '2013-12-31'
  AND term_code = 'cash';

4. Отобразите имя, фамилию и названия аккаунтов людей в поле `network_username`, у которых названия аккаунтов начинаются на `'Silver'`.

In [ ]:
SELECT
       first_name,
       last_name,
       network_username
FROM people
WHERE network_username LIKE 'Silver%';

5. Выведите на экран всю информацию о людях, у которых названия аккаунтов в поле `network_username` содержат подстроку `'money'`, а фамилия начинается на `'K'`.

In [ ]:
SELECT *
FROM people
WHERE network_username LIKE '%money%' AND last_name LIKE 'K%';

6. Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

In [ ]:
SELECT
      SUM(funding_total),
      country_code
FROM company
GROUP BY country_code
ORDER BY SUM(funding_total) DESC;

7. Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [ ]:
SELECT
       funded_at,
       MIN(raised_amount) AS min_amount,
       MAX(raised_amount) AS max_amount
FROM funding_round
GROUP BY funded_at
HAVING MIN(raised_amount) > 0 AND MIN(raised_amount) != MAX(raised_amount);

8. Создайте поле с категориями:
- Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию `high_activity`.
- Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию `middle_activity`.
- Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию `low_activity`.

Отобразите все поля таблицы fund и новое поле с категориями.

In [ ]:
SELECT *,
    CASE
        WHEN invested_companies >= 100 THEN 'high_activity'
        WHEN invested_companies >= 20 AND invested_companies < 100 THEN 'middle_activity'
        ELSE 'low_activity'
    END AS category
FROM fund;

9. Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
SELECT activity,
       ROUND(AVG(investment_rounds)) AS avg_investment_rounds
FROM (
    SELECT *,
           CASE
               WHEN invested_companies >= 100 THEN 'high_activity'
               WHEN invested_companies >= 20 THEN 'middle_activity'
               ELSE 'low_activity'
           END AS activity
    FROM fund
) AS categorized_fund
GROUP BY activity
ORDER BY avg_investment_rounds ASC;

10. Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы.

Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю.

Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке.

In [ ]:
SELECT country_code,
       MIN(invested_companies) AS min_invested_companies,
       MAX(invested_companies) AS max_invested_companies,
       AVG(invested_companies) AS avg_invested_companies
FROM fund
WHERE founded_at BETWEEN '2010-01-01' AND '2012-12-31'
GROUP BY country_code
HAVING MIN(invested_companies) != 0
ORDER BY avg_invested_companies DESC, country_code
LIMIT 10;

11. Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [ ]:
SELECT
       p.first_name,
       p.last_name,
       e.instituition
FROM people AS p
LEFT JOIN education AS e ON p.id = e.person_id;

12. Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

In [ ]:
SELECT
       c.name,
       COUNT(DISTINCT e.instituition) AS count_institution
FROM company AS c
JOIN people AS p ON c.id = p.company_id
JOIN education AS e ON p.id = e.person_id
GROUP BY c.name
ORDER BY count_institution DESC
LIMIT 5;

13. Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [ ]:
SELECT DISTINCT c.name
FROM company AS c
JOIN funding_round AS fr ON c.id = fr.company_id
JOIN (
    SELECT company_id
    FROM funding_round
    WHERE is_first_round = 1 AND is_last_round = 1
) AS first_last_rounds ON c.id = first_last_rounds.company_id
WHERE c.status = 'closed';

14. Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [ ]:
SELECT DISTINCT p.id
FROM company AS c
JOIN funding_round AS fr ON c.id = fr.company_id
JOIN people AS p ON c.id = p.company_id
WHERE c.status = 'closed'
  AND c.id IN (
    SELECT company_id
    FROM funding_round
    WHERE is_first_round = 1 AND is_last_round = 1
  );

15. Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
WITH closed_companies AS (
    SELECT id
    FROM company
    WHERE status = 'closed'
),
first_last_rounds AS (
    SELECT company_id
    FROM funding_round
    WHERE is_first_round = 1 AND is_last_round = 1
)
SELECT DISTINCT p.id, e.instituition
FROM closed_companies c
JOIN first_last_rounds fr ON c.id = fr.company_id
JOIN people p ON c.id = p.company_id
JOIN education e ON p.id = e.person_id;

16. Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

In [ ]:
SELECT p.id AS person_id,
       COUNT(e.instituition) AS num_institutions
FROM people AS p
JOIN education AS e ON e.person_id = p.id
WHERE p.company_id IN (
    SELECT c.id
    FROM company AS c
    JOIN funding_round AS fr ON fr.company_id = c.id
    WHERE fr.is_first_round = 1
      AND fr.is_last_round = 1
      AND c.status = 'closed'
)
GROUP BY p.id;

17. Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [ ]:
SELECT AVG(num_institutions) AS avg_num_institutions
FROM (
    SELECT p.id AS person_id,
           COUNT(e.instituition) AS num_institutions
    FROM people AS p
    JOIN education AS e ON e.person_id = p.id
    WHERE p.company_id IN (
        SELECT c.id
        FROM company AS c
        JOIN funding_round AS fr ON fr.company_id = c.id
        WHERE fr.is_first_round = 1
          AND fr.is_last_round = 1
          AND c.status = 'closed'
    )
    GROUP BY p.id
) AS subquery;

18. Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Socialnet.

In [ ]:
SELECT AVG(num_institutions) AS avg_num_institutions
FROM (
    SELECT p.id AS person_id,
           COUNT(e.instituition) AS num_institutions
    FROM people AS p
    JOIN education AS e ON e.person_id = p.id
    WHERE p.company_id = (
        SELECT c.id
        FROM company AS c
        WHERE c.name = 'Socialnet'
    )
    GROUP BY p.id
) AS subquery;

19. Составьте таблицу из полей:
- name_of_fund — название фонда;
- name_of_company — название компании;
- amount — сумма инвестиций, которую привлекла компания в раунде.

В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
SELECT
       f.name AS name_of_fund,
       c.name AS name_of_company,
       fr. raised_amount AS amount
FROM investment AS i
    JOIN company AS c ON i.company_id = c.id
    JOIN fund AS f ON i.fund_id = f.id
    JOIN funding_round AS fr ON i.funding_round_id = fr.id
WHERE c.milestones > 6
AND EXTRACT(YEAR FROM fr.funded_at) BETWEEN 2012 AND 2013;

20. Выгрузите таблицу, в которой будут такие поля:
- название компании-покупателя;
- сумма сделки;
- название компании, которую купили;
- сумма инвестиций, вложенных в купленную компанию;
- доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.

Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы.

Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

In [ ]:
SELECT
    c.name AS name_of_acquiring,
    a.price_amount AS amount,
    c_1.name AS name_of_acquired,
    c_1.funding_total AS total_funding,
    ROUND(a.price_amount / NULLIF(c_1.funding_total, 0)) AS ratio
FROM acquisition AS a
LEFT JOIN company AS c ON a.acquiring_company_id = c.id
LEFT JOIN company AS c_1 ON a.acquired_company_id = c_1.id
WHERE a.price_amount > 0 AND c_1.funding_total > 0
ORDER BY amount DESC, name_of_acquired
LIMIT 10;

21. Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

In [ ]:
SELECT
       c.name,
       EXTRACT(MONTH FROM CAST(fr.funded_at AS date)) AS month
FROM company AS c
JOIN funding_round AS fr ON c.id = fr.company_id
WHERE c.category_code = 'social'
  AND fr.funded_at BETWEEN '2010-01-01' AND '2013-12-31'
  AND fr.raised_amount > 0;

22. Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
- номер месяца, в котором проходили раунды;
- количество уникальных названий фондов из США, которые инвестировали в этом месяце;
- количество компаний, купленных за этот месяц;
- общая сумма сделок по покупкам в этом месяце.

In [ ]:
WITH funds AS (
    SELECT
           EXTRACT(MONTH FROM CAST(fr.funded_at AS date)) AS month,
           COUNT(DISTINCT f.id) AS count_funds
    FROM fund AS f
    JOIN investment AS i ON f.id = i.fund_id
    JOIN funding_round AS fr ON i.funding_round_id = fr.id
    WHERE fr.funded_at BETWEEN '2010-01-01' AND '2013-12-31'
    AND f.country_code = 'USA'
    GROUP BY month
),

acquired AS (
    SELECT
           EXTRACT(MONTH FROM CAST(a.acquired_at AS date)) AS month,
           COUNT(a.id) AS acquired_companies,
           SUM(a.price_amount) AS amount
    FROM acquisition AS a
    WHERE a.acquired_at BETWEEN '2010-01-01' AND '2013-12-31'
    GROUP BY month
)

SELECT
       funds.month,
       funds.count_funds,
       acquired.acquired_companies,
       acquired.amount
FROM funds
JOIN acquired ON funds.month=acquired.month;

23. Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
WITH
    inv_2011 AS (
        SELECT country_code,
            AVG(funding_total) AS avg_investment_2011
        FROM company
        WHERE EXTRACT(YEAR FROM founded_at) = 2011
        GROUP BY country_code
    ),
    inv_2012 AS (
        SELECT country_code,
            AVG(funding_total) AS avg_investment_2012
        FROM company
        WHERE EXTRACT(YEAR FROM founded_at) = 2012
        GROUP BY country_code
    ),
    inv_2013 AS (
        SELECT country_code,
            AVG(funding_total) AS avg_investment_2013
        FROM company
        WHERE EXTRACT(YEAR FROM founded_at) = 2013
        GROUP BY country_code
    )
SELECT
    inv_2011.country_code AS country_code_2011,
    inv_2011.avg_investment_2011,
    inv_2012.avg_investment_2012,
    inv_2013.avg_investment_2013
FROM inv_2011
JOIN inv_2012 ON inv_2011.country_code = inv_2012.country_code
JOIN inv_2013 ON inv_2011.country_code = inv_2013.country_code
ORDER BY inv_2011.avg_investment_2011 DESC;